In [1]:
# Scraper 설치
!pip install pyLDAvis # LDA 시각화
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git # 한국어 띄어쓰기 패키지(pykospacing)
!pip install git+https://github.com/ssut/py-hanspell.git # 한국어 Norm : 맞춤법 교정(hanspell)

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh
!pip install konlpy

! pip install MeCab
# 오류나면 [런타임 다시 시작] 누르고 다시 설치 필요 없이 진행 !!!!!

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 87.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-50bpbcgb
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-50bpbcgb
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit 5bb0b3e8f62c1936a430745297821a9ecd78beb1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 kB 29.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing backend dependencies

In [1]:
# Data 불러오기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/OTT_team_project

/content/drive/MyDrive/Colab Notebooks/OTT_team_project


In [3]:
import utils.LDA_utils as lda_utils

## 전처리 필요 라이브러리
import re
import time
import datetime
import numpy as np
import pandas as pd

# 단어 임베딩
from sklearn.manifold import TSNE
from gensim import models
import itertools

# 기타
from tqdm import tqdm
import warnings # 경고 메시지 무시
import matplotlib.pyplot as plt

warnings.filterwarnings(action='ignore')
plt.rcParams['font.family'] = 'Malgun Gothic'

## Main

Topic modeling

In [4]:
# OTT 선택, 전체로 할 시 None 으로 기입
# OTT = 'tiving'
OTT = None

# 토픽 결정 수
num_topics = 5

# 불용어 
add_stopwords_list = ['티빙', '넷플릭스', '웨이브', '쿠팡', '쿠팡플레이', '디즈니', '나무',
                      '디즈니플러스', '애플리케이션', '개발자', '도도', '슈슈', '휴휴',
                      '옥수수', '라프텔', '라프']

timeslices=[4097, 16623, 3051]

In [5]:
# 데이터 불러오기
df = pd.read_csv('./data/review_data.csv', encoding='utf-8-sig')
if OTT: 
  df_ott = df[df['ott']== OTT]
else:
  df_ott = df.copy()


### 리뷰 기간 조정 및 결측치 처리 ###
df_ott = lda_utils.date_extract(df_ott, start_day='2021-11-12', printing=True)

2021: 5044 rows

2022: 18833 rows

2023: 6684 rows



In [10]:
for o in df_ott.ott.unique():
  print(o, len(df_ott[df_ott['ott']==o]))

tiving 4810
disney 6774
netflix 5608
wave 7433
coupang 5936


In [ ]:
# 데이터 불러오기
df = pd.read_csv('./data/review_data.csv', encoding='utf-8-sig')
if OTT: 
  df_ott = df[df['ott']== OTT]
else:
  df_ott = df.copy()


### 리뷰 기간 조정 및 결측치 처리 ###
df_ott = lda_utils.date_extract(df_ott, start_day='2021-11-12', printing=True)

### 토픽모델링 ###
token_data = lda_utils.Data_processing(df_ott) # 전처리 시작
tokenized_data, _ = token_data.get_token(add_stopwords_list)
lda = lda_utils.Model(tokenized_data, num_topics=num_topics) # 토픽모델링 시작
LDAseq = lda.LDAseqmodel(timeslices=timeslices)

In [ ]:
# 학습된 토픽들과 해당하는 단어들 출력
for t in range(3):
    topics = LDAseq.print_topics(time=t)
    print("---------------")
    for topic in topics:
        print(topic)

topic_table

Word Embedding